In [1]:
import os
import re
import json
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

# List your YouTube URLs here
YOUTUBE_URLS = [
    "https://www.youtube.com/watch?v=xH_IlU7nIRs&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=1",			
"https://www.youtube.com/watch?v=HGWRLRgOm8E&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=2",			
"https://www.youtube.com/watch?v=3X5x0UJ5EyI&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=3",			
"https://www.youtube.com/watch?v=xDPM2Py7ALw&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=4",			
"https://www.youtube.com/watch?v=GoKFRoalQNA&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=5",			
"https://www.youtube.com/watch?v=RuXAm3tyc48&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=6",			
"https://www.youtube.com/watch?v=83jHrKbi2tI&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=7",			
"https://www.youtube.com/watch?v=KCSCrA4X3Vw&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=8",			
"https://www.youtube.com/watch?v=7mo167ohvJw&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=9",			
"https://www.youtube.com/watch?v=bdhVtONQAfM&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=10",			
"https://www.youtube.com/watch?v=k-RTEIaYvAg&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=11",			
"https://www.youtube.com/watch?v=cnW_HDIBmz4&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=12",			
"https://www.youtube.com/watch?v=LKxOamnP8J4&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=13",			
"https://www.youtube.com/watch?v=vTTzDjdfILs&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=14",			
"https://www.youtube.com/watch?v=szUhmDH98oQ&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=15",			
"https://www.youtube.com/watch?v=MqqXTrEEZ7Y&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=16",			
"https://www.youtube.com/watch?v=DJQrJuiVK60&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=17",			
"https://www.youtube.com/watch?v=qQLLDPAEzT0&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=18",			
"https://www.youtube.com/watch?v=VcDCRPWgnbM&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=19",			
"https://www.youtube.com/watch?v=n4YoZDQs6VA&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=20",			
"https://www.youtube.com/watch?v=wYPee7aczTE&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=21",			
"https://www.youtube.com/watch?v=J9lTE95LPaA&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=22",			
"https://www.youtube.com/watch?v=8yfM_EttsZw&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=23",			
"https://www.youtube.com/watch?v=d6R35yGrsgQ&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=24",			
"https://www.youtube.com/watch?v=jhvCc5tKAno&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=25",			
"https://www.youtube.com/watch?v=cWEiafGi4QE&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=26",			
"https://www.youtube.com/watch?v=yuLbtRS2j8U&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=27",			
"https://www.youtube.com/watch?v=_US1NnuGP4I&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=28",			
"https://www.youtube.com/watch?v=MFCdA2vGVh4&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=29",			
"https://www.youtube.com/watch?v=CBJ3_A5SMsc&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=30",			
"https://www.youtube.com/watch?v=ZB7jJCdfQX8&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=31",			
"https://www.youtube.com/watch?v=ZhMEUikbVIk&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=32",			
"https://www.youtube.com/watch?v=SAjZj3ZwebA&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=33",			
"https://www.youtube.com/watch?v=2m-18YW7yf4&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=34",			
"https://www.youtube.com/watch?v=tEha8BzsAZ8&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=35",			
"https://www.youtube.com/watch?v=wkpFiOh88qQ&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=36",   			
"https://www.youtube.com/watch?v=zPE-s4Xbv4U&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=37",			
"https://www.youtube.com/watch?v=OT9l855nxgg&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=38",			
"https://www.youtube.com/watch?v=7I6MTgRxg00&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=39",			
"https://www.youtube.com/watch?v=olWhTwaYP_Y&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=40",			
"https://www.youtube.com/watch?v=E2dzSPOhUOI&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=41",			
"https://www.youtube.com/watch?v=boklbFhF8l8&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=42",			
"https://www.youtube.com/watch?v=5rkw5Pz57bI&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=43",			
"https://www.youtube.com/watch?v=9TtKUydRfDM&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=44",			
"https://www.youtube.com/watch?v=3ULlocLanC8&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=45",			
"https://www.youtube.com/watch?v=YoOsvcxLy40&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=46",			
"https://www.youtube.com/watch?v=nzIAe8WSSqE&list=PLY67ac_8iF3lLCkHb9IF9WG47xr3c9nOt&index=47"			
]

OUTPUT_DIR = "transcripts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def extract_video_id(url):
    """
    Extract the video ID from a YouTube URL.
    Handles URLs like:
    - https://www.youtube.com/watch?v=VIDEO_ID
    - https://youtu.be/VIDEO_ID
    """
    # Parse the URL
    parsed_url = urlparse(url)

    if parsed_url.hostname in ["www.youtube.com", "youtube.com"]:
        query = parse_qs(parsed_url.query)
        video_id = query.get("v")
        if video_id:
            return video_id[0]
    elif parsed_url.hostname == "youtu.be":
        return parsed_url.path[1:]
    return None

def fetch_and_save_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        # Combine text parts into one string (with spaces)
        full_text = " ".join([entry['text'] for entry in transcript_list])

        # Save plain text transcript
        txt_path = os.path.join(OUTPUT_DIR, f"{video_id}.txt")
        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(full_text)

        # Save full transcript JSON with timestamps (optional)
        json_path = os.path.join(OUTPUT_DIR, f"{video_id}_with_timestamps.json")
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(transcript_list, f, indent=2)

        print(f"✅ Transcript saved for video {video_id}")

    except TranscriptsDisabled:
        print(f"❌ Transcripts are disabled for video {video_id}")
    except NoTranscriptFound:
        print(f"❌ No transcript found for video {video_id}")
    except Exception as e:
        print(f"❌ Error fetching transcript for video {video_id}: {e}")

def main():
    for url in YOUTUBE_URLS:
        video_id = extract_video_id(url)
        if video_id:
            print(f"Fetching transcript for video ID: {video_id}")
            fetch_and_save_transcript(video_id)
        else:
            print(f"❌ Could not extract video ID from URL: {url}")

if __name__ == "__main__":
    main()



Fetching transcript for video ID: xH_IlU7nIRs
✅ Transcript saved for video xH_IlU7nIRs
Fetching transcript for video ID: HGWRLRgOm8E
✅ Transcript saved for video HGWRLRgOm8E
Fetching transcript for video ID: 3X5x0UJ5EyI
✅ Transcript saved for video 3X5x0UJ5EyI
Fetching transcript for video ID: xDPM2Py7ALw
✅ Transcript saved for video xDPM2Py7ALw
Fetching transcript for video ID: GoKFRoalQNA
✅ Transcript saved for video GoKFRoalQNA
Fetching transcript for video ID: RuXAm3tyc48
✅ Transcript saved for video RuXAm3tyc48
Fetching transcript for video ID: 83jHrKbi2tI
✅ Transcript saved for video 83jHrKbi2tI
Fetching transcript for video ID: KCSCrA4X3Vw
✅ Transcript saved for video KCSCrA4X3Vw
Fetching transcript for video ID: 7mo167ohvJw
✅ Transcript saved for video 7mo167ohvJw
Fetching transcript for video ID: bdhVtONQAfM
✅ Transcript saved for video bdhVtONQAfM
Fetching transcript for video ID: k-RTEIaYvAg
✅ Transcript saved for video k-RTEIaYvAg
Fetching transcript for video ID: cnW_HDIBm